# 6a. MLlib Introduction

Provides tools like:

* ML Algorithms
* Featurization: feature extraction, transformation, ...
* Pipelines
* Persistence: saving and loading algorithms
* Utilities: linear algebra, statistics, data handling

_Note: RDD-based APIs are considered in maintenance mode. DataFrame-based API is primary_

## Dependencies

MLlib uses linear algebra packages fro optimised numerical processing, which may call native acceleration libraries that are required. However, native acceleration libraries cannot be distributed together with Spark. Will see a warning message if it is not used

For Python you just need NumPy >=1.4

---

# Basic Statistics

MLlib is able to perform basic (and complex) statistics on large data

## Correlation

Provide the flexibility to calculate pairware correlation among many series. Includes Pearson's and Spearman's correlation algorithms

Output here is the correlation matrix for the input Dataset of Vectors using the specified method.

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName("mllib") \
  .getOrCreate()

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]), ), 
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]), ), 
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]), ), 
				(Vectors.sparse(4, [(0, 9.0), (3, 1.0)]), ),
        ]
df = spark.createDataFrame(data, ["features"])
r1 = Correlation.corr(df, 'features').head()
print(f"Pearson correlation matrix:\n {str(r1[0])}")

r2 = Correlation.corr(df, 'features', 'spearman').head()
print(f"Spearman correlation matrix:\n {str(r2[0])}")

Pearson correlation matrix:
 DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])
Spearman correlation matrix:
 DenseMatrix([[1.        , 0.10540926,        nan, 0.4       ],
             [0.10540926, 1.        ,        nan, 0.9486833 ],
             [       nan,        nan, 1.        ,        nan],
             [0.4       , 0.9486833 ,        nan, 1.        ]])


## Hypothesis Testing

Determine whether a result is statistically significant. Only supports Pearson's Chi-squared tests for independence.

Chi-squared tests: for each feature and label pair, they are converted into a contingency matrix and computed. All labels and features must be categorical

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ['label', 'features'])

r = ChiSquareTest.test(df, 'features', 'label').head()

print("pValues: " + str(r.pValues))
print(f'degrees of freedom {r.degreesOfFreedom}')
print(f"Statistics: {str(r.statistics)}")

pValues: [0.6872892787909721,0.6822703303362126]
degrees of freedom [2, 3]
Statistics: [0.75,1.5]


# Summarizer

Vector column summary statistics (max, min, mean, median, std, variance, etc)

In [6]:
from pyspark.ml.stat import Summarizer
from pyspark.sql import Row
from pyspark.ml.linalg import Vector

from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("appName").setMaster('master')
sc = SparkContext(conf=conf)

df = sc.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
                     Row(weight=0.0, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()
									
# Create summarizer for multiple metrics 'mean' and 'count'
summarizer = Summarizer.metrics('mean', 'count')

# Compute statistics for multiple metrics with weight
df.select(summarizer.summary(df.features, df.weight)).show(truncate=False)

# Compute statistics for multiple metrics without weight
df.select(summarizer.summary(df.features)).show(truncate=False)

# Compute statistics for single metric 'mean' with weight
df.select(summarizer.mean(df.features, df.weight)).show(truncate=False)

# Compute statistics for single metrics 'mean' without weight
df.select(summarizer.mean(df.features)).show(truncate=False)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=mllib, master=local[*]) created by getOrCreate at C:\Users\Martin Ho\AppData\Local\Temp\ipykernel_21920\2451862296.py:5 

---

# Data Source

How to use data source in ML to load data. Introduce 2 types of data sources beside the general data sources (e.g Parquet, CSV, JSON, etc)

1. Image data source
2. LIBSVM data source

## Image data source

Load image files from a directory. Can eb compressed images into raw representation. DataFrame has 1 `StructType` column: "image" containing image data stored as a schema. The Schema:

* origin: StringType (represents the file path of the image)
* height: IntegerType (height of the image)
* width: IntegerType (width of the image)
* nChannels: IntegerType (number of image channels)
* mode: IntegerType (OpenCV-compatible type)
* data: BinaryType (Image bytes in OpenCV-compatible order: row-wise BGR in most cases)

Will usually provide Spark SQL data source API to load images as a DataFrame

In [ ]:
df = spark.read.format('image').option('dropInvalid', True).load('data/mllib/images/origin/kittens')
df.select('image.origin', 'image.width', 'image.height').show(truncate=False)

## LIBSVM data source

Used to load 'libsvm' type files from a directory. Contains 2 columns: 

* label: DoubleType (represents the instance label)
* features: VectorUDT (represents the feature vector)

In [8]:
df = spark.read.format('libsvm').option('numFeatures', '780').load('data/mllib/sample_libsvm_data.txt')
df.show(10)

---

# Pipelines

Provide a uniform set of high-level APIs built on top of DataFrames to help users create and tune practical machine learning pipelines.

## Main Concept

Makes it easier to combine multiple algorithms into a single pipeline or workflow. Pipelines are mostly inspired by scikit-learn projects

* `DataFrame`: Uses `DataFrame` from Spark SQL as the ML dataset which can hold different data types
* `Transformer`: Transforms one `DataFrame` into another (e.g transforming a `DataFrame` from features into one with predictions)
* `Estimator`: Algorithm that's fit on a `DataFrame` to produce a `Transformer` (e.g learning algorithm is the `Estimator` that trains and produces a model) 
* `Pipelines`: Chain multiple of the above into an ML workflow
* `Parameter`: Common API for specifying parameters

### DataFrame

* `DataFrame` supports many different basic and structured data types from Spark SQL. 
* Can also use `Vector` type. 
* Can be created implicitly or explicitly from a regular RDD. 
* Columns are also named

## Pipeline Components

### Transformers

Abstraction that includes feature transformation and learned models. Generally it implements the method `transform()` that appends new column(s). For example:

* Read a column, map it to a new column and output a new `DataFrame` with the mapped column appended
* read the column containing feature vectors and predict the label for each feature vector

### Estimators

Abstracts the concept of a learning algorithm or any algorithm that fits or trains data. Implements a method `fit()` which accepts data and produces a model (that is a `Transformer`)

## Properties of pipeline components

`Transformer.transform()` and `Estimators.fit()` are stateless. Each instance has a unique ID which is useful in specifying the parameters assocaited

## Pipeline

MLlib representations of workflows, a sequence of `PipelineStages` to be run in order.

__How it works__

* Each stage is either a `Transformer` or `Estimator`
* Input data is transformed as it passes through each stage
	- `Tranformer.transform()`
	- `Estimator.fit()` -> creates a model -> `Transform.transform()`
* All `Estimators` in the original Pipeline will become `Transformers` once the model has been fitted
* Each stages `transform()` method passes the newly formed dataset onto the next stage